In [351]:
# region General Imports
import os
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

from model_optim.model_optimizer import ModelOptimizer

# Dataset
from custom_datasets.fatigue_mi import FatigueMI

%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of model_optim.model_optimizer failed: Traceback (most recent call last):
  File "/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 274, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    object.__setatt

In [352]:
model_optimizer = ModelOptimizer(
    dataset=FatigueMI(),
    model_name="shallow_conv_net"
)

In [356]:
model_optimizer.clean_up(
    type="remove_all_but_best_trial_data",
    subjects=[2],
)

['./temp/[2]/6405d11e654b42aca9df48458c67ecde/model/study_best_trial.npy']

Found 1 best trials for subjects: [2]

In [220]:
study = model_optimizer.search_best_model(
    subjects=[2],
    # channels=[],
    max_iter = 50,
    max_epochs=5,
    max_stag_count=25,
    rounds=1,
    replace_previous_study_for_subjects=True
)

/home/arazzz/Documents/GitHub/moabb_model_optimization/model_optimization_tests/model_optim/model_optimizer.py:366: ExperimentalWarning: NSGAIIISampler is experimental (supported from v3.2.0). The interface can change in the future.
  )


  0%|          | 0/50 [00:00<?, ?it/s]

Adding metadata with 3 columns
Epoch 1/5


/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.4706 - loss: 116.1792 - val_accuracy: 0.6667 - val_loss: 95.3579 - learning_rate: 0.0010
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6324 - loss: 95.4247 - val_accuracy: 0.5000 - val_loss: 83.6081 - learning_rate: 0.0010
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7794 - loss: 83.1752 - val_accuracy: 0.5000 - val_loss: 74.5729 - learning_rate: 0.0010
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7647 - loss: 74.2643 - val_accuracy: 0.5000 - val_loss: 67.8358 - learning_rate: 0.0010
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8235 - loss: 67.1407 - val_accuracy: 0.6111 - val_loss: 61.3621 - learning_rate: 0.0010


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 669ms/step - accuracy: 0.5000 - loss: 61.4992


Adding metadata with 3 columns
Epoch 1/5


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5735 - loss: 93.0541 - val_accuracy: 0.6111 - val_loss: 78.9845 - learning_rate: 0.0010
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.4853 - loss: 79.3279 - val_accuracy: 0.5000 - val_loss: 70.1522 - learning_rate: 0.0010
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7059 - loss: 70.1247 - val_accuracy: 0.7222 - val_loss: 63.5631 - learning_rate: 0.0010
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6176 - loss: 63.7565 - val_accuracy: 0.5000 - val_loss: 58.2236 - learning_rate: 0.0010
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6765 - loss: 58.1775 - val_accuracy: 0.5000 - val_loss: 53.7027 - learning_rate: 0.0010


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step - accuracy: 0.4545 - loss: 53.7726


Sampling frequency of the instance is already 300.0, returning unmodified.
Adding metadata with 3 columns
Epoch 1/5


/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5147 - loss: 210.7533 - val_accuracy: 0.5000 - val_loss: 163.4747 - learning_rate: 0.0010
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.7500 - loss: 162.5758 - val_accuracy: 0.5000 - val_loss: 144.2014 - learning_rate: 0.0010
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.6176 - loss: 138.2003 - val_accuracy: 0.5000 - val_loss: 125.2474 - learning_rate: 0.0010
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.6618 - loss: 119.2954 - val_accuracy: 0.5000 - val_loss: 111.7038 - learning_rate: 0.0010
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.8088 - loss: 104.4307 - val_accuracy: 0.5556 - val_loss: 95.1938 - learning_rate: 0.0010


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 783ms/step - accuracy: 0.5000 - loss: 95.7429


Sampling frequency of the instance is already 300.0, returning unmodified.
Adding metadata with 3 columns
Epoch 1/5


/home/arazzz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5202 - loss: 129.3267 - val_accuracy: 0.4444 - val_loss: 108.3611 - learning_rate: 0.0010
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5450 - loss: 102.2371 - val_accuracy: 0.4444 - val_loss: 90.0805 - learning_rate: 0.0010
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6051 - loss: 88.3105 - val_accuracy: 0.6667 - val_loss: 78.5619 - learning_rate: 0.0010
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7898 - loss: 75.6530 - val_accuracy: 0.5000 - val_loss: 69.3219 - learning_rate: 0.0010
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9550 - loss: 66.3131 - val_accuracy: 0.6667 - val_loss: 61.5035 - learning_rate: 0.0010


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step - accuracy: 0.5455 - loss: 63.2556




In [221]:
model_optimizer.get_study_metrics(study).sort_values(by="scores", ascending=True)
# model_optimizer.get_study_metrics(study).sort_values(by="test_acc", ascending=False)

,train_acc,test_acc,val_acc,train_val_diff,scores,channels_selected
10,0.941176,0.636364,0.888889,0.052288,0.111611,"[P3, F3, C4, P4, Pz, O2, F7, F8, A2, T6]"
30,0.808824,0.409091,0.722222,0.086601,0.278128,"[P3, P4, T5, O2, F8, T6, T4]"
33,0.705882,0.454545,0.722222,0.016340,0.278278,"[P3, Fz, C4, P4, Cz, Pz, T5, O2, F8, T6]"
20,0.985294,0.454545,0.722222,0.263072,0.278328,"[P3, C3, F3, Fz, C4, P4, Cz, Fp1, Fp2, T3, O2]"
35,0.955882,0.545455,0.666667,0.289216,0.333783,"[P3, C3, F3, Fz, F4, P4, Fp2, O2, T4]"
12,0.897059,0.454545,0.666667,0.230392,0.333883,"[P3, C3, F4, C4, Pz, Fp2, T3, O1, O2, F7, F8]"
32,0.882353,0.363636,0.666667,0.215686,0.333933,"[C3, F4, C4, P4, Pz, Fp2, T5, O2, F7, F8, T6, T4]"
0,0.823529,0.500000,0.666667,0.156863,0.333933,"[C3, F3, Cz, Pz, Fp1, T3, T5, O2, F8, A2, T6, T4]"
25,0.838235,0.454545,0.666667,0.171569,0.333983,"[P3, Fz, F4, C4, P4, Pz, Fp1, T3, T5, O2, F7, ..."
18,0.661765,0.590909,0.666667,0.004902,0.333983,"[C3, F3, Fz, P4, Cz, Pz, Fp2, T5, O2, F7, A2, ..."


In [355]:
rpprint({ k: v for k, v in study.best_trial.params.items() if not k.startswith("channels") })
rprint("test_accuracy =", study.best_trial.user_attrs["trial_data"]["test_accuracy"])
rprint("val_accuracy =", np.max(study.best_trial.user_attrs["trial_data"]["val_accuracy"]))
rprint("channels_selected =", study.best_trial.user_attrs["trial_data"]["channels_selected"])

{
│   'sfreq': 256,
│   'batch_size': 224,
│   'pool_size_d2': 45,
│   'strides_d2': 28,
│   'conv_filters_d2': 43,
│   'conv2d_1_units': 140,
│   'conv2d_2_units': 160,
│   'l2_reg_1': 0.761906790459666,
│   'l2_reg_2': 0.7499413031314208,
│   'l2_reg_3': 0.5216769006644256,
│   'dropout_rate': 0.7000000000000001
}

test_accuracy = 0.6363636255264282

val_accuracy = 0.8888888955116272

channels_selected = ['P3' 'F3' 'C4' 'P4' 'Pz' 'O2' 'F7' 'F8' 'A2' 'T6']